In [22]:
#Bibs
from mpi4py import MPI
import numpy as np

N = 1000000


In [33]:
%%writefile serial.py

import numpy as np
N = 1000000


def MonteCarlo(x, y):
    n = x.shape[0]
    total_acerto = 0
    for i in range(n):

        if x[i]**2 + y[i]**2 <= 1.0 :
            total_acerto +=1

    return float(total_acerto / n)

x = np.random.rand((N))
y = np.random.rand((N))

print("Resultado de pi: {:.10f}".format(4.0*MonteCarlo(x,y)))

Overwriting serial.py


In [34]:
!time python3 serial.py

Resultado de pi: 3.1397080000

real	0m0,393s
user	0m0,507s
sys	0m0,432s


In [46]:
%%writefile reduce.py


import numpy as np
from mpi4py import MPI


def MonteCarlo(x, y):
    n = x.shape[0]
    total_acerto = 0
    for i in range(n):

        if x[i]**2 + y[i]**2 <= 1.0 :
            total_acerto +=1

    return float(total_acerto / n)


comm = MPI.COMM_WORLD

size = comm.Get_size()
rank = comm.Get_rank()

N = 1000000
'''Inicializa Vetor com N posicoes'''
x = np.random.rand((N//size))
y = np.random.rand((N//size))

half_pi = MonteCarlo(x,y)

soma = comm.reduce(half_pi, op=MPI.SUM, root=0)

if rank == 0:
    print("total de ranks = {}".format(size))
    print("Rank = {}".format(rank))
    print("Valor obtido de Pi = {:.8f}".format(soma))




Overwriting reduce.py


In [45]:
!time mpirun -n 4 python3 reduce.py


real	0m0,003s
user	0m0,003s
sys	0m0,000s
